In [9]:
import requests
from PIL import Image
from io import BytesIO
import sys

# Function to fetch a satellite image from Google Maps API
def fetch_image(latitude, longitude, zoom=18, size=(640, 640), maptype='satellite', api_key='YOUR_API_KEY'):
    url = f"https://maps.googleapis.com/maps/api/staticmap?center={latitude},{longitude}&zoom={zoom}&size={size[0]}x{size[1]}&maptype={maptype}&key={api_key}"
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    return image


In [23]:
with open('api_key.txt', 'r') as file:
    api_key = file.read()
# Location for st.kidane mihret church
latitude, longitude = (9.105704174639172, 38.646911907819145)
zoom = 18
image = fetch_image(latitude, longitude, zoom=zoom, api_key = api_key)

In [24]:
# Check the image
image.show()

In [3]:
from ultralytics import YOLO

model = YOLO('/Users/declanbracken/Development/Khthon/code/Computer_Vision/models/v1/best.pt')

# Function to preprocess images and perform inference
def detect_churches(image):
    results = model(image)
    # boxes = results[0].boxes
    return results

In [14]:
import numpy as np

# Function to generate grid coordinates
def generate_grid(center_lat, center_lon, area_km, img_size=0.36): # 360 meters in image width corresponding to zoom 18
    # This is a rough conversion of area to lat/lon
    lat_range = area_km / 111.32  # 1 degree of latitude is approximately 111.32 km
    lon_range = area_km / (111.32 * np.cos(center_lat * np.pi / 180))

    lat_img = img_size / 111.32
    lon_img = img_size / (111.32 * np.cos(center_lat * np.pi / 180))

    latitudes = np.arange(center_lat - lat_range/2, center_lat + lat_range/2, lat_img)
    longitudes = np.arange(center_lon - lon_range/2, center_lon + lon_range/2, lon_img)
    
    grid_coords = [(lat, lon) for lat in latitudes for lon in longitudes]
    return grid_coords

zoom_to_area = {18: 0.36, 19: 0.179}

# Example usage to generate grid coordinates
api_key = ''
center_lat, center_lon = (9.105704174639172, 38.646911907819145)  # Example coordinates
area_km = 1
zoom = 18
img_size = zoom_to_area[zoom]
grid_coords = generate_grid(center_lat, center_lon, area_km, img_size=img_size)
# print(grid_coords)
# Perform detection on each grid point
detections = []
confidence = []
for coord in grid_coords:
    image = fetch_image(coord[0], coord[1], api_key=api_key, zoom = zoom)
    results = detect_churches(image)
    detections.append(results)



0: 640x640 (no detections), 175.1ms
Speed: 2.7ms preprocess, 175.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


In [15]:
result = detections[0]
result[0].show()

In [20]:
boxes = detect_churches(image)


0: 640x640 1 octagon-church, 105.1ms
Speed: 2.6ms preprocess, 105.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [24]:
print(boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.])
conf: tensor([0.6806])
data: tensor([[306.5905, 313.8973, 335.4470, 339.4632,   0.6806,   1.0000]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([1, 6])
xywh: tensor([[321.0187, 326.6802,  28.8564,  25.5659]])
xywhn: tensor([[0.5016, 0.5104, 0.0451, 0.0399]])
xyxy: tensor([[306.5905, 313.8973, 335.4470, 339.4632]])
xyxyn: tensor([[0.4790, 0.4905, 0.5241, 0.5304]])
